In [17]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from astropy.io import fits
import os
import datetime
from matplotlib.colors import LogNorm
from pybirales.birales import BiralesFacade
from pybirales.birales_config import BiralesConfig
from pybirales.pipeline.modules.detection.dbscan_detection import _validate, _create, detect, partition_input_data, dbscan_clustering,create_clusters 
from pybirales.pipeline.modules.detection.exceptions import DetectionClusterIsNotValid
from pybirales.pipeline.modules.detection.queue import BeamCandidatesQueue
from pybirales.pipeline.modules.detection.space_debris_candidate import SpaceDebrisTrack
from scipy import stats
from pybirales import settings

matplotlib.rcParams['figure.figsize'] = (12, 10)

2018-04-11 17:55:02,380 DEBUG    12975 MainProcess     MainThread           backend module://ipykernel.pylab.backend_inline version unknown


In [2]:
# Global variables
HOME_DIR = os.environ['HOME']
FITS_DIR = os.path.join(HOME_DIR, '.birales/visualisation/fits/')
FITS_FILENAME = 'Observation_2018-04-11T1010/Observation_2018-04-11T1010_filtered.fits'
BEAM_ID = 4
OBS_INFO = {
    'start_center_frequency':410.0703125,
    'channel_bandwidth':9.5367431640625e-06,
    'transmitter_frequency': 410.105,
    'timestamp': datetime.datetime(2017, 6, 30, 12, 13, 30, 455859),
    'sampling_time':0.1048576,
    'nchans': 8192,
}
BIRALES_CONFIG = os.path.join(HOME_DIR, '.birales/configuration/birales.ini')
DETECTION_CONFIG = os.path.join(HOME_DIR, '.birales/configuration/detection.ini')

In [3]:
# Load the BIRALES configuration from file
config = BiralesConfig([BIRALES_CONFIG, DETECTION_CONFIG], {})

# Initialise the Birales Facade (BOSS)
bf = BiralesFacade(configuration=config)



2018-04-11 17:47:27,666 INFO     12975 MainProcess     MainThread           Loading configuration files
2018-04-11 17:47:27,668 INFO     12975 MainProcess     MainThread           Loading the /home/denis/.birales/configuration/birales.ini configuration file.
2018-04-11 17:47:27,672 INFO     12975 MainProcess     MainThread           Loaded the /home/denis/.birales/configuration/birales.ini configuration file.
2018-04-11 17:47:27,674 INFO     12975 MainProcess     MainThread           Loading the /home/denis/.venv/birales/local/lib/python2.7/site-packages/pybirales/configuration/backend/roach_backend.ini configuration file.
2018-04-11 17:47:27,676 INFO     12975 MainProcess     MainThread           Loaded the /home/denis/.venv/birales/local/lib/python2.7/site-packages/pybirales/configuration/backend/roach_backend.ini configuration file.
2018-04-11 17:47:27,683 INFO     12975 MainProcess     MainThread           Configurations successfully loaded.
2018-04-11 17:47:27,685 INFO     12975 M

In [4]:
filtered_fits_filepath = os.path.join(FITS_DIR, FITS_FILENAME)
input_data = fits.getdata(filtered_fits_filepath)
print ('Filtered image shape:', input_data.shape)

('Filtered image shape:', (1, 8192, 288))


In [5]:
t0 = np.datetime64(OBS_INFO['timestamp'])
td = np.timedelta64(int(OBS_INFO['sampling_time'] * 1e9), 'ns')
channels = np.arange(OBS_INFO['start_center_frequency'], OBS_INFO['start_center_frequency'] + OBS_INFO['channel_bandwidth'] * OBS_INFO[
                                          'nchans'],
                                      OBS_INFO['channel_bandwidth'])

In [10]:
# Run the detection algorithm

# Process a slice of the input data
beam_ndx, power_data = partition_input_data(input_data, 0)

print beam_ndx.shape

# Run the clustering algorithm on the beam data
labelled_data, power_data = dbscan_clustering(beam_ndx, power_data)

unique_clusters = np.unique(labelled_data[:, 2]).tolist()
print 'Detected {} unique clusters'.format(len(unique_clusters))

(3569, 2)
Detected 12 unique clusters


In [20]:
for label in unique_clusters:
    cluster_data = labelled_data[labelled_data[:, 2] == label]

    channel_ndx = cluster_data[:, 0]
    time_ndx = cluster_data[:, 1]
    
    m, _, r_value, _, _ = stats.linregress(channel_ndx, time_ndx)
    
    if channel_ndx.shape[0] < 3:
        print 'Rejected {} because too small (n={})'.format(label, channel_ndx.shape[0])
        continue
    
    if np.abs(r_value) < settings.detection.linearity_thold:
        print 'Rejected {} because not linear (r={:0.2f})'.format(label, r_value)
        continue
        
    print 'Cluster {} is OK (n={})'.format(label, channel_ndx.shape[0])
    
    label_mask = labelled_data[:, 2] == label
    valid, cluster = _create(power_data[label_mask], labelled_data[label_mask], channels, t0, td, 0, 1)
    
    print valid, cluster
    

print create_clusters(power_data, labelled_data, channels, t0, td, 0, 1)

Rejected 0 because not linear (r=0.85)
Rejected 1 because not linear (r=0.91)
Cluster 2 is OK (n=29)
False None
Rejected 3 because not linear (r=0.00)
Cluster 4 is OK (n=5)
False None
Rejected 5 because not linear (r=-0.91)
Rejected 6 because not linear (r=0.39)
Rejected 7 because not linear (r=0.65)
Rejected 8 because not linear (r=-0.70)
Rejected 9 because not linear (r=0.17)
Rejected 10 because not linear (r=0.80)
Rejected 11 because not linear (r=0.63)
[]
